# Evaluators Validation Notebook 
Este notebook demonstra, **cálculo a cálculo**, como devem ser obtidas as
pontuações de cada métrica usada em *pyAutoSummarizer* **sem** importar as
classes prontas de `evaluation/`.  


In [1]:
## Dependências essenciais
# !pip -q install rouge-score nltk sentence-transformers gensim
import warnings, nltk
warnings.filterwarnings('ignore', category=FutureWarning)
nltk.download('punkt_tab', quiet=True) # Baixa o PunktTokenizer

True

## 1  Lexical

> Precisaremos aplicar ``StopWords`` + ``Stemming`` antes de aplicar cada Evaluator Lexical

> Nele, precisaremos flaguear se as aplicações serão em ``PT`` ou ``EN``, únicas transformações disponíveis inicialmente

**Exemplo**
- Referência: “Eu amo estudar na Universidade Federal Fluminense.”  
- Resumo gerado: “Eu adoro estudar na UFF.”

Calcularemos três métricas tradicionais:
1. ROUGE‑1 F1  
2. ROUGE‑L F1  
3. BLEU‑4  
4. METEOR

📝 **Formato esperado** — um `dict` com quatro chaves (`rouge1_f1`,
`rougeL_f1`, `bleu4`, `meteor`) e valores `float ∈ [0,1]`.

In [6]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score

reference = "Eu amo estudar na Universidade Federal Fluminense."
generated  = "Eu adoro estudar na UFF."

# ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True) 
# Aqui é aplicado um stemmer em inglês, 
# Para aplicarmos nosso caso de texto ptbr, preciaríamos usar o RSLPSTemmer 
rouge_scores = scorer.score(reference, generated)
lexical = {
    'rouge1_f1': rouge_scores['rouge1'].fmeasure,
    'rougeL_f1': rouge_scores['rougeL'].fmeasure,
}

# BLEU‑4
chen = SmoothingFunction()
lexical['bleu4'] = sentence_bleu(
    [reference.split()], # Token de Reference 
    generated.split(), # Token de Generado
    weights=(0.25,0.25,0.25,0.25), # Pesos para BLEU-4 (4 pesos pois Bleu4)
    smoothing_function=chen.method1 # Evita que o produto vire 0
)

# METEOR
lexical['meteor'] = meteor_score([reference.split()], generated.split())

print(lexical)


{'rouge1_f1': 0.5, 'rougeL_f1': 0.5, 'bleu4': 0.08428828344718171, 'meteor': 0.3758169934640523}


### Tratando Dados para uma Avaliação Correta (PTBR)

> Precisamos remover ``stopwords`` e ``stemming`` para ter uma avaliação mais fiel a realidade

In [ ]:
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer, SnowballStemmer
nltk.download('rslp',quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download("punkt", quiet=True)




def preprocess_text(text: str,LANGUAGE='pt') -> str:
    if LANGUAGE == "pt":
        _STOP_SET  = set(stopwords.words("portuguese"))
        _STEMMER   = RSLPStemmer()
        _TOKEN_LANG = "portuguese"
    else:                          # english
        _STOP_SET  = set(stopwords.words("english"))
        # Porter é o mais leve; Snowball costuma ser ligeiramente melhor
        _STEMMER   = SnowballStemmer("english")
        _TOKEN_LANG = "english"
    

    def stemming(generated):
        tokens = [_STEMMER.stem(w) for w in generated.split()]
        texto_stemmed = " ".join(tokens)
        return texto_stemmed

    def remove_stopwords(text: str) -> str:
        return " ".join(
            w for w in word_tokenize(text, language=_TOKEN_LANG)
            if w.lower() not in _STOP_SET
        )

    def remove_ruido(texto: str) -> str:
        _URLS_HASHTAGS_MENCOES = re.compile(r'https?://\S+|www\.\S+|[@#]\w+', flags=re.UNICODE)
        _DIGITOS                = re.compile(r'\d+', flags=re.UNICODE)
        _PONTUACAO_E_SIMBOLOS   = re.compile(r'[^A-Za-zÀ-ÖØ-öø-ÿ\s]', flags=re.UNICODE)
        _ESPACOS                = re.compile(r'\s+', flags=re.UNICODE)

        # 1) URLs, hashtags, @menções
        texto = _URLS_HASHTAGS_MENCOES.sub(' ', texto)

        # 2) Dígitos
        texto = _DIGITOS.sub(' ', texto)

        # 3) Pontuação e símbolos 
        texto = _PONTUACAO_E_SIMBOLOS.sub(' ', texto)

        # 4) Espaços múltiplos
        texto = _ESPACOS.sub(' ', texto).strip()

        # 7. Caixa baixa (consistência)
        return texto.lower()

    text = remove_ruido(text)
    text = remove_stopwords(text)
    text = stemming(text)
    return text

In [37]:
reference = "A quick brown fox jumps over the lazy dog."
generated  = "A fast brown fox leaps over a lazy dog."
preprocess_text(reference,LANGUAGE='english')

'quick brown fox jump lazi dog'

In [20]:
reference = "Eu amo estudar na Universidade Federal Fluminense."
generated  = "Eu adoro estudar na UFF."
reference_preprocessed = preprocess_text(reference)
generated_preprocessed  = preprocess_text(generated)
# ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True) 
# Aqui é aplicado um stemmer em inglês, 
# Para aplicarmos nosso caso de texto ptbr, preciaríamos usar o RSLPSTemmer 
rouge_scores = scorer.score(reference_preprocessed, generated_preprocessed)
lexical = {
    'rouge1_f1': rouge_scores['rouge1'].fmeasure,
    'rougeL_f1': rouge_scores['rougeL'].fmeasure,
}

# BLEU‑4
chen = SmoothingFunction()
lexical['bleu4'] = sentence_bleu(
    [reference_preprocessed.split()], # Token de Reference 
    generated_preprocessed.split(), # Token de Generado
    weights=(0.25,0.25,0.25,0.25), # Pesos para BLEU-4 (4 pesos pois Bleu4)
    smoothing_function=chen.method1 # Evita que o produto vire 0
)

# METEOR
lexical['meteor'] = meteor_score([reference_preprocessed.split()], generated_preprocessed.split())

In [22]:
print(reference_preprocessed,generated_preprocessed,lexical)

amo estud univers feder flumin ador estud uff {'rouge1_f1': 0.25, 'rougeL_f1': 0.25, 'bleu4': 0.05833544737207805, 'meteor': 0.10416666666666666}


## 2  Semantic

Agora mediremos similaridade de significado usando **Sentence‑BERT** . Escolhemos uma frase de exemplo em inglês
porque os modelos pré‑treinados são mais robustos nesse idioma.

### Sentence BERT

> Faz sentido aplicar para parágrafos completos ? Deveria quebrar por sentenças ?

* Podemos gerar um embedding represetnando as senteças e comparar entre si
* Podemos considerar tudo um só sentença 

In [39]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

# Two lists of sentences
sentences1 = [
    "The new movie is awesome",
    "The cat sits outside",
    "A man is playing guitar",
]

sentences2 = [
    "The dog plays in the garden",
    "The new movie is so great",
    "A woman watches TV",
]

# Compute embeddings for both lists
embeddings1 = model.encode(sentences1)
embeddings2 = model.encode(sentences2)

# Compute cosine similarities
similarities = model.similarity(embeddings1, embeddings2)

# Output the pairs with their score
for idx_i, sentence1 in enumerate(sentences1):
    print(sentence1)
    for idx_j, sentence2 in enumerate(sentences2):
        print(f" - {sentence2: <30}: {similarities[idx_i][idx_j]:.4f}")

The new movie is awesome
 - The dog plays in the garden   : 0.0543
 - The new movie is so great     : 0.8939
 - A woman watches TV            : -0.0502
The cat sits outside
 - The dog plays in the garden   : 0.2838
 - The new movie is so great     : -0.0029
 - A woman watches TV            : 0.1310
A man is playing guitar
 - The dog plays in the garden   : 0.2277
 - The new movie is so great     : -0.0136
 - A woman watches TV            : -0.0327


### BERT Score 

**Representação de Tokens:** 
    O BERTScore utiliza embeddings contextuais (provenientes de modelos pré-treinados como BERT, RoBERTa, XLNet ou XLM) para representar os tokens nas sentenças de entrada. A vantagem desses embeddings é que eles geram representações vetoriais diferentes para a mesma palavra em sentenças distintas, dependendo do contexto circundante.

**Medida de Similaridade:** 
    A similaridade entre um token $x_i$ da referência e um token $\hat{x}_j$ da candidata é calculada pela similaridade de cosseno entre seus respectivos vetores de embedding. Essa abordagem oferece uma medida de similaridade "suave", superando a necessidade de correspondência exata de strings.

**Correspondência Greedy:** Para calcular a precisão e o recall, o BERTScore emprega uma estratégia de correspondência gulosa (greedy matching). Isso significa que cada token é correspondido ao token mais similar na outra sentença para maximizar a pontuação de similaridade.

**Cálculo das Pontuações (Precision, Recall, F1):**

> Recall (R_BERT): É a média das similaridades de cosseno máximas, onde cada token da referência é correspondido ao token mais similar na candidata 

> Precision (P_BERT): É a média das similaridades de cosseno máximas, onde cada token da candidata é correspondido ao token mais similar na referência.

> F1 (F_BERT): É a média harmônica da Precisão e do Recall. De forma geral, a recomendação é usar F1, pois ele se mostra mais confiável em diversos cenários.

In [51]:
import bert_score
reference = "A quick brown fox jumps over the lazy dog."
generated  = "A fast brown fox leaps over a lazy dog."
lang = 'en'  # Define the language for BERTScore
P, R, F1 = bert_score.score([generated], [reference], lang=lang,
                             rescale_with_baseline=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


https://github.com/Tiiiger/bert_score#readme

In [52]:
print(P,R,F1)

tensor([0.8388]) tensor([0.8388]) tensor([0.8391])


## 3  Factual  
*(métricas serão adicionadas futuramente)*